In [1]:
import os

In [2]:
%pwd

'/home/ubuntu/learning/mlops/pacmann/lazada-id-reviews/notebooks'

In [3]:
# Change to the main directory
# So, it's executed from main directory
os.chdir("../")

In [4]:
%pwd

'/home/ubuntu/learning/mlops/pacmann/lazada-id-reviews'

### Data Preprocessing Config

This code will be apply in `src/LadazaIDReview/entity/config_entity.py`

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataDumpConfig:
    root_dir: Path
    reviews_path: Path
    input_train_path: Path
    input_test_path: Path
    output_train_path: Path
    output_test_path: Path
    params_test_size: float

@dataclass(frozen=True)
class DataPreprocessingConfig:
    root_dir: Path
    input_train_path: Path
    input_test_path: Path
    vectorized_train_path: Path
    vectorized_test_path: Path
    model_dir: Path
    vectorizer_model_path: Path

### Data Preprocessing Config Manager

This code will be apply in `src/LazadaIDReview/config/configurations.py`.

In [6]:
from LazadaIDReviews.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from LazadaIDReviews.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_dump_data_config(self) -> DataDumpConfig:
        """read data dump config file and store as config entity
        then apply the dataclasses
        
        Returns:
            config: PreprocessingConfig type
        """
        dump_config = self.config.dump_data
        ingest_config = self.config.ingest_from_sql
        dataset_params = self.params

        create_directories([dump_config.root_dir])

        config = DataDumpConfig(
            root_dir=dump_config.root_dir,
            reviews_path=ingest_config.reviews_path,
            input_train_path=dump_config.input_train_path,
            input_test_path=dump_config.input_test_path,
            output_train_path=dump_config.output_train_path,
            output_test_path=dump_config.output_test_path,
            params_test_size=dataset_params.TEST_SIZE
        )

        return config
    
    def get_preprocessing_data_config(self) -> DataPreprocessingConfig:
        """read preprocessing config file and store as config entity
        then apply the dataclasses
        
        Returns:
            config: PreprocessingConfig type
        """
        dump_config = self.config.dump_data
        vectorize_config = self.config.vectorize_data
        train_config = self.config.train_model

        create_directories([vectorize_config.root_dir])

        config = DataPreprocessingConfig(
            root_dir=vectorize_config.root_dir,
            input_train_path=Path(dump_config.input_train_path),
            input_test_path=Path(dump_config.input_test_path),
            vectorized_train_path=Path(vectorize_config.vectorized_train_path),
            vectorized_test_path=Path(vectorize_config.vectorized_test_path),
            model_dir=train_config.root_dir,
            vectorizer_model_path=Path(vectorize_config.vectorizer_model_path)
        )

        return config

### Perform Preprocessing

This code in `src/LazadaIDReview/components/preprocessing.py`.

What we would do?
+ Drop null values
+ Splitting the dataset to train and test data
+ Vectorize text using `TFIDF`

As stated before; let’s load, select columns, and drop null values from dataset.

In [9]:
import joblib
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from imblearn.over_sampling import RandomOverSampler

from LazadaIDReviews import logger

class Preprocessing:
    def __init__(self, config: DataDumpConfig):
        self.config = config

    def dump_data(self) -> None:
        """dump the splited dataset to data training and testing
        """
        logger.info(f"Read reviews file.")
        dataset_reviews = pd.read_csv(self.config.reviews_path)
        dataset = dataset_reviews[["rating", "reviewContent"]].copy()
        dataset.dropna(inplace=True)
        
        logger.info(f"Split reviews file to data train and test.")
        X_train, X_test, y_train, y_test = train_test_split(
            dataset["reviewContent"], 
            dataset["rating"], 
            test_size=self.config.params_test_size
        )
        
        # NOTE: improve the performance with ROS
        logger.info(f"Perform random over sampler.")
        ros = RandomOverSampler()
        X_train_ros, y_train_ros = ros.fit_resample(pd.DataFrame(X_train), pd.DataFrame(y_train))
        
        # NOTE: data save as series
        logger.info(f"Dump data train into {self.config.root_dir} directory.")
        X_train_ros["reviewContent"].to_pickle(self.config.input_train_path)
        X_test.to_pickle(self.config.input_test_path)
        
        # NOTE: data save as series
        logger.info(f"Dump data test into {self.config.root_dir} directory.")
        y_train_ros["rating"].to_pickle(self.config.output_train_path)
        y_test.to_pickle(self.config.output_test_path)
        
    def vectorize_data(self) -> None:
        """vectorize the splited dataset and dump vectorizer model
        """
        vectorizer = TfidfVectorizer()
        
        logger.info(f"Load data train in {self.config.input_train_path}.")
        X_train = joblib.load(self.config.input_train_path)
        
        logger.info(f"Load data test in {self.config.input_test_path}.")
        X_test = joblib.load(self.config.input_test_path)
        
        logger.info(f"Vectorize the data.")
        X_train_vec = vectorizer.fit_transform(X_train)
        X_test_vec = vectorizer.transform(X_test)
        
        logger.info(f"Dump the vectorized data.")
        joblib.dump(X_train_vec, self.config.vectorized_train_path)
        joblib.dump(X_test_vec, self.config.vectorized_test_path)
        
        logger.info(f"Creating {self.config.model_dir} directory.")
        model_dir = str(self.config.model_dir)
        os.makedirs(model_dir, exist_ok=True)
        
        logger.info(f"Save the vectorizer model.")
        joblib.dump(vectorizer, self.config.vectorizer_model_path)  

### Dump the Data Train and Data Test

This code in `src/LazadaIDReview/pipeline/step_02_preprocessing.py`.

In [10]:
try:
    config = ConfigurationManager()
    dump_data_config = config.get_dump_data_config()
    data_ingestion = Preprocessing(config=dump_data_config)
    data_ingestion.dump_data()
except Exception as e:
    logger.error(e)
    raise e

[2024-09-11 04:31:46,469: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-11 04:31:46,479: INFO: common: yaml file: metrics/params.yaml loaded successfully]
[2024-09-11 04:31:46,481: INFO: common: created directory at: artifacts]
[2024-09-11 04:31:46,482: INFO: common: created directory at: artifacts/data]
[2024-09-11 04:31:46,484: INFO: 2608593555: Read reviews file.]
[2024-09-11 04:31:47,187: INFO: 2608593555: Split reviews file to data train and test.]
[2024-09-11 04:31:47,195: INFO: 2608593555: Perform random over sampler.]
[2024-09-11 04:31:47,207: INFO: 2608593555: Dump data train into artifacts/data directory.]
[2024-09-11 04:31:47,254: INFO: 2608593555: Dump data test into artifacts/data directory.]


**Debug**: Read data

In [11]:
X_train = joblib.load(dump_data_config.input_train_path)
X_train

0        Barangnya baru sampai, baru pertama kali di ke...
1                    Tv nya jossss, 3 hari nyampe cikarang
2               Barang nya bagus gan..mantap lah pokok nya
3        Produk telah tiba sesuai estimasi.. pelayanan ...
4                                                    bagus
                               ...                        
82950             Mau kredit. Cicilan nya berapa x bayar ?
82951    eliminasi 2 hari packing rapi tapi warna gak s...
82952    TV nya sdh sampe dan bagus... cuma lama banget...
82953    bagus hanya saja saat disatukan layar dengan k...
82954    puas bgt kalo untuk tv ini, harga murah...kual...
Name: reviewContent, Length: 82955, dtype: object

In [12]:
X_train.isnull().sum()

0

In [13]:
y_train = joblib.load(dump_data_config.output_train_path)
y_train

0        1
1        5
2        5
3        5
4        5
        ..
82950    4
82951    4
82952    4
82953    4
82954    4
Name: rating, Length: 82955, dtype: int64

In [14]:
X_test = joblib.load(dump_data_config.input_test_path)
X_test

4101              Terimakasih barang udah sampe dan sesuai.
139337    makasih banget buat Nadira Store dn lazada.psa...
201703    Bagus, sesuai dengan pesanan... Sangat puas pe...
30999     Produknya bagus, sesuai dengan apa yang di des...
93353                                        Sesuai pesanan
                                ...                        
172426                                   mantap dah... imut
110691    Barang yang dikirim sesuai dengan deskripsi, p...
45451     produk nya bagus .. cm pas pengiriman nya tanp...
83813                                                👏🏻👏🏻👏🏻
145913                                        Mantul betttt
Name: reviewContent, Length: 85624, dtype: object

In [15]:
X_test.isnull().sum()

0

In [16]:
y_test = joblib.load(dump_data_config.output_test_path)
y_test

4101      4
139337    5
201703    5
30999     5
93353     5
         ..
172426    5
110691    5
45451     5
83813     5
145913    5
Name: rating, Length: 85624, dtype: int64

### Vectorize the Data Train and Data Test

This code in `src/LazadaIDReview/pipeline/step_02_preprocessing.py`.

In [17]:
try:
    config = ConfigurationManager()
    preprocessing_config = config.get_preprocessing_data_config()
    preprocessing = Preprocessing(config=preprocessing_config)
    preprocessing.vectorize_data()
except Exception as e:
    logger.error(e)
    raise e

[2024-09-11 04:32:10,322: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-09-11 04:32:10,324: INFO: common: yaml file: metrics/params.yaml loaded successfully]
[2024-09-11 04:32:10,325: INFO: common: created directory at: artifacts]
[2024-09-11 04:32:10,326: INFO: common: created directory at: artifacts/preprocessing]
[2024-09-11 04:32:10,327: INFO: 2608593555: Load data train in artifacts/data/X_train.pkl.]
[2024-09-11 04:32:10,404: INFO: 2608593555: Load data test in artifacts/data/X_test.pkl.]
[2024-09-11 04:32:10,527: INFO: 2608593555: Vectorize the data.]
[2024-09-11 04:32:12,311: INFO: 2608593555: Dump the vectorized data.]
[2024-09-11 04:32:12,399: INFO: 2608593555: Creating artifacts/models directory.]
[2024-09-11 04:32:12,399: INFO: 2608593555: Save the vectorizer model.]


**Debug**: Read data

In [18]:
X_train_vec = joblib.load(preprocessing_config.vectorized_train_path)
X_train_vec

<82955x13588 sparse matrix of type '<class 'numpy.float64'>'
	with 1224387 stored elements in Compressed Sparse Row format>

In [19]:
X_test_vec = joblib.load(preprocessing_config.vectorized_test_path)
X_test_vec

<85624x13588 sparse matrix of type '<class 'numpy.float64'>'
	with 981246 stored elements in Compressed Sparse Row format>